## DuckDuckGo Tool

DuckDuckGo is a privacy-focused search engine that emphasizes user anonymity and does not track its users. Its search API provides a way for developers to leverage its search capabilities programmatically. This can be particularly valuable in applications where preserving user privacy is a concern.

In this context, many developers might be interested in using the DuckDuckGo search capabilities combined with Python, a versatile programming language, within the LangChain framework. LangChain refers to a framework designed for developing applications that utilize language models. It allows for the integration of various components, such as APIs and data management systems, into cohesive applications.

In [21]:
from duckduckgo_search import DDGS

In this function, we define the search_duckduckgo function that takes a query string and an optional max_results parameter to limit the number of results returned. We then utilize the ddg method from the duckduckgo_search package to get the results and print them out.

In [13]:
def search_duckduckgo(query, max_results=5):
    ddgs = DDGS()  # Create the DDGS instance first
    results = ddgs.text(query, max_results=max_results)  # Then call the search method
    for i, result in enumerate(results):
        print(f"{i+1}: {result['title']} - {result['href']}")

if __name__ == "__main__":
    search_duckduckgo("What is acne?")

1: Acne - Symptoms and causes - Mayo Clinic - https://www.mayoclinic.org/diseases-conditions/acne/symptoms-causes/syc-20368047
2: Acne: Treatment, Types, Causes, Prevention, and More - Healthline - https://www.healthline.com/health/skin/acne
3: Acne: Types, Causes, Treatment & Prevention - Cleveland Clinic - https://my.clevelandclinic.org/health/diseases/12233-acne
4: What is Acne? Definition & Types | NIAMS - National Institute of ... - https://www.niams.nih.gov/health-topics/acne
5: Acne Causes: What Is Acne and Why Do I Have It? - WebMD - https://www.webmd.com/skin-problems-and-treatments/acne/understanding-acne-basics


Connecting our .env file which has api which we can't share or show in our main code and to link it with our code we use load_dotenv

In [15]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

## Integrating DuckDuckGo with LangChain: Setting Up Your Chain
Now that you can perform a simple DuckDuckGo search, we will integrate this functionality into the LangChain framework. LangChain introduces the concept of ‘chains,’ which allows for the creation of pipelines that can integrate various components, such as retrieval, transformation, and output.

To set up a basic LangChain integration, we first need to define our DuckDuckGo search as part of a chain.

In [17]:
import os
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI

# Define your search prompt template
search_prompt_template = PromptTemplate.from_template(
    "Find information about {topic}. Provide a summary of the top results."
)

def gemini_search_chain(query):
    # Initialize the Gemini model using API key from environment variable
    # No need to set the API key explicitly as it's already in your environment
    gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    
    # Create a chain with the prompt template and Gemini model
    chain = LLMChain(llm=gemini, prompt=search_prompt_template)
    
    # Run the chain with the query
    result = chain.run(topic=query)
    
    return result

# Example usage
if __name__ == "__main__":
    result = gemini_search_chain("Large Language Models")
    print(result)

Okay, here's a summary of information about Large Language Models (LLMs), compiled from what I consider to be the top results (based on authority, relevance, and clarity):

**What are Large Language Models (LLMs)?**

Large Language Models (LLMs) are a type of artificial intelligence (AI) model that are trained on massive amounts of text data to understand, generate, and manipulate human language. They are characterized by their:

*   **Size:**  They have a huge number of parameters (billions or even trillions), which allows them to learn complex patterns in language.
*   **Training Data:**  Trained on vast datasets of text and code scraped from the internet, books, articles, and other sources.
*   **Transformer Architecture:**  Almost all modern LLMs are based on the transformer architecture, which enables them to process information in parallel and capture long-range dependencies in text.
*   **Emergent Abilities:**  LLMs exhibit surprising emergent abilities, meaning capabilities tha

In the above code, we define a prompt template, which allows you to structure your queries for the model. The duckduckgo_search_chain function formats the prompt by substituting the {topic} with the desired query.

You would then pass the formatted prompt to a language model of your choice within LangChain (here represented as Gemini), which can provide summaries, responses, or knowledge based on the search results.

## Retrieving and Summarizing Results
Supposing you have integrated the search functionality into a LangChain application, the next step is to automate the retrieval and summarization of search results. After searching DuckDuckGo for a given query, you want to leverage the language model to summarize the results.

In [18]:
# Define your search prompt template
search_prompt_template = PromptTemplate.from_template(
    "Summarize the following search results about {topic}:\n\n{results}"
)

def duckduckgo_search_and_summarize(query, max_results=5):
    # Perform DuckDuckGo search
    ddgs = DDGS()
    results = ddgs.text(query, max_results=max_results)
    
    # Format search results for the summary
    formatted_results = ""
    for i, result in enumerate(results):
        formatted_results += f"{i+1}. {result['title']}: {result['href']}\n{result.get('body', '')}\n\n"
    
    # Initialize the Gemini model using API key from environment variables
    gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    
    # Create a chain with the prompt template and Gemini model
    chain = LLMChain(llm=gemini, prompt=search_prompt_template)
    
    # Run the chain with the search results
    summary = chain.run(topic=query, results=formatted_results)
    
    return summary

# Example usage
if __name__ == "__main__":
    query = "LangChain framework"
    summary = duckduckgo_search_and_summarize(query)
    print(f"Summary for '{query}':")
    print(summary)

Summary for 'LangChain framework':
LangChain is an open-source framework designed to simplify the development, productionization, and deployment of applications powered by Large Language Models (LLMs). It facilitates the integration of LLMs into various use cases like document analysis, chatbots, and code analysis. The framework is composable and can be used standalone or integrated with other LangChain products like LangGraph (for agentic workflows) and LangSmith (for agent evaluation and observability). It provides tools for building context-aware reasoning applications and helps developers manage and scale their LLM applications.


In the above function, we perform the DuckDuckGo search, compile the titles and links of the results into a summary, and format that summary for a final query to the language model, which would be responsible for generating a polished output.

## Error Handling and Enhancements
In a real-world application, you may encounter various errors such as connection issues, invalid queries, or unexpected API responses. Effective error handling is crucial for maintaining a robust application. Below is an example of incorporating basic error handling in the search function.

In [19]:
def safe_search_duckduckgo(query):
    try:
        # Create DDGS instance first
        ddgs = DDGS()
        # Then call the text search method
        results = ddgs.text(query)
        
        if not results:
            return "No results found."
        return results
    except Exception as e:
        return f"An error occurred: {str(e)}"

The above function captures exceptions that may arise during the API call and provides user-friendly feedback. It is advisable to expand on this by implementing logging mechanisms for better monitoring and maintenance.

Additionally, consider optimizing your queries by adding parameters, such as specifying the region or the type of information you’re looking for to enhance user experience.

## Now we are doing the compile up work of above code creating a chatbot with whom we can ask question based on the summary provided by it.

In [20]:
import os
from dotenv import load_dotenv
from duckduckgo_search import DDGS
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables from .env file
load_dotenv()

# Define your prompt templates
search_prompt_template = PromptTemplate.from_template(
    "Summarize the following search results about {topic}:\n\n{results}"
)

follow_up_prompt_template = PromptTemplate.from_template(
    "Based on the previous information about {original_topic}, answer this follow-up question: {follow_up_question}\n\n"
    "Previous summary: {previous_summary}\n\n"
    "If you need additional information to answer the follow-up, here are new search results: {new_results}"
)

def safe_search_duckduckgo(query, max_results=5):
    """
    Safely perform a DuckDuckGo search and handle any errors.
    """
    try:
        # Create DDGS instance first
        ddgs = DDGS()
        # Then call the text search method
        results = ddgs.text(query, max_results=max_results)
        
        if not results:
            return "No results found."
        return results
    except Exception as e:
        return f"An error occurred: {str(e)}"

def format_search_results(results):
    """
    Format search results into a readable string.
    """
    if isinstance(results, str):
        return results  # This is an error message
    
    formatted_results = ""
    for i, result in enumerate(results):
        formatted_results += f"{i+1}. {result['title']}: {result['href']}\n{result.get('body', '')}\n\n"
    
    return formatted_results

def duckduckgo_search_and_summarize(query, max_results=5):
    """
    Search DuckDuckGo and summarize the results using Gemini.
    """
    # Perform DuckDuckGo search
    results = safe_search_duckduckgo(query, max_results)
    
    # Format search results
    formatted_results = format_search_results(results)
    
    # Initialize the Gemini model
    gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    
    # Create a chain with the prompt template and Gemini model
    chain = LLMChain(llm=gemini, prompt=search_prompt_template)
    
    # Run the chain with the search results
    summary = chain.run(topic=query, results=formatted_results)
    
    return summary, formatted_results

def handle_follow_up(follow_up_question, original_query, previous_summary, previous_results):
    """
    Handle follow-up questions by searching again if needed and using context.
    """
    # First, try to see if we need new information
    combined_query = f"{original_query} {follow_up_question}"
    
    # Do a new search for the follow-up
    new_results = safe_search_duckduckgo(combined_query)
    formatted_new_results = format_search_results(new_results)
    
    # Initialize the Gemini model
    gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    
    # Create a chain with the follow-up prompt template
    follow_up_chain = LLMChain(llm=gemini, prompt=follow_up_prompt_template)
    
    # Run the chain with all available context
    response = follow_up_chain.run(
        original_topic=original_query,
        follow_up_question=follow_up_question,
        previous_summary=previous_summary,
        new_results=formatted_new_results
    )
    
    return response, formatted_new_results

def conversational_handler():
    """
    Main function to handle the conversation flow.
    """
    print("Welcome to the Conversational Search System!")
    print("Ask a question, and I'll search for information and summarize it.")
    print("Type 'exit' at any time to quit.\n")
    
    original_query = None
    previous_summary = None
    previous_results = None
    
    while True:
        # First query or new topic
        if not original_query:
            query = input("\nWhat would you like to search for? ")
            if query.lower() == 'exit':
                break
                
            print("\nSearching and summarizing...")
            summary, results = duckduckgo_search_and_summarize(query)
            
            print("\n--- SEARCH SUMMARY ---")
            print(summary)
            
            original_query = query
            previous_summary = summary
            previous_results = results
        
        # Follow-up questions
        follow_up = input("\nWhat else would you like to know about this topic? (or type 'new' for a new search, 'exit' to quit) ")
        
        if follow_up.lower() == 'exit':
            break
        elif follow_up.lower() == 'new':
            original_query = None
            previous_summary = None
            previous_results = None
            continue
        
        print("\nProcessing your follow-up question...")
        follow_up_response, new_results = handle_follow_up(
            follow_up, 
            original_query, 
            previous_summary, 
            previous_results
        )
        
        print("\n--- FOLLOW-UP RESPONSE ---")
        print(follow_up_response)
        
        # Update the previous summary to include this follow-up
        previous_summary += f"\n\nFollow-up about {follow_up}:\n{follow_up_response}"
        previous_results += new_results

if __name__ == "__main__":
    conversational_handler()

Welcome to the Conversational Search System!
Ask a question, and I'll search for information and summarize it.
Type 'exit' at any time to quit.


Searching and summarizing...

--- SEARCH SUMMARY ---
Large Language Models (LLMs) are AI systems, specifically machine learning models, designed for natural language processing. They utilize deep learning techniques and neural networks with a massive number of parameters, trained on vast amounts of text data using self-supervised learning. This allows them to understand, generate, summarize, and predict text-based content. LLMs are a type of generative AI and are incredibly flexible, capable of performing diverse tasks like question answering, summarization, translation, and sentence completion. They have the potential to revolutionize content creation, search engines, and virtual assistants. Some of the most powerful LLMs are based on the Generative Pre-trained Transformer (GPT) architecture.

Processing your follow-up question...

--- FOLLO

# Below is the code explanation

# Complete Explanation of the Conversational Search System
Let me walk through the entire codebase, explaining each component and how they work together to create a conversational search experience.


## Imports and Setup

import os <br>
from dotenv import load_dotenv <br>
from duckduckgo_search import DDGS <br>
from langchain.prompts import PromptTemplate <br>
from langchain.chains import LLMChain <br>
from langchain_google_genai import ChatGoogleGenerativeAI <br>

#Load environment variables from .env file <br>
load_dotenv()<br>


This section:

->Imports the necessary libraries for environment variables, DuckDuckGo search, and LangChain components<br>
->Uses load_dotenv() to load your Gemini API key from a .env file into the environment variables<br>

## Prompt Templates

search_prompt_template = PromptTemplate.from_template( <br>
    "Summarize the following search results about {topic}:\n\n{results}"
)<br>

follow_up_prompt_template = PromptTemplate.from_template( <br>
    "Based on the previous information about {original_topic}, answer this follow-up question: {follow_up_question}\n\n" <br>
    "Previous summary: {previous_summary}\n\n" <br>
    "If you need additional information to answer the follow-up, here are new search results: {new_results}"
)<br>
These templates define how we'll instruct the Gemini model:

->search_prompt_template: Used for initial queries to summarize search results <br>
->follow_up_prompt_template: Used for follow-up questions, providing context from previous interactions

## Function: safe_search_duckduckgo

def safe_search_duckduckgo(query, max_results=5): <br>
    """ <br>
    Safely perform a DuckDuckGo search and handle any errors.<br>
    """<br>
    try:<br>
        # Create DDGS instance first<br>
        ddgs = DDGS() <br>
        # Then call the text search method <br>
        results = ddgs.text(query, max_results=max_results) <br>
        
        if not results: 
            return "No results found." 
        return results 
    except Exception as e:
        return f"An error occurred: {str(e)}" 


Role: This function is a wrapper around the DuckDuckGo search API that:

->Creates a DDGS client <br>
->Performs a text search with the given query <br>
->Handles potential errors gracefully <br>
->Returns either search results or an error message if the search fails <br>
->Limits results to a specified number (default 5) <br>

## Function: format_search_results <br>
def format_search_results(results): <br>
    """<br>
    Format search results into a readable string. <br>
    """<br>
    if isinstance(results, str):<br>
        return results  # This is an error message<br>
    
    formatted_results = ""
    for i, result in enumerate(results):
        formatted_results += f"{i+1}. {result['title']}: {result['href']}\n{result.get('body', '')}\n\n"
    
    return formatted_results

Role: This function takes raw search results and converts them into a formatted string:

->Checks if the input is already a string (error message)<br>
->Creates a numbered list of search results<br>
->Includes the title, URL, and body text (if available) for each result<br>
->Separates results with newlines for readability<br>

## Function: duckduckgo_search_and_summarize<br>
def duckduckgo_search_and_summarize(query, max_results=5):<br>
    """<br>
    Search DuckDuckGo and summarize the results using Gemini.<br>
    """<br>
    # Perform DuckDuckGo search<br>
    results = safe_search_duckduckgo(query, max_results)<br>
    
    # Format search results 

    formatted_results = format_search_results(results)
    
    # Initialize the Gemini model

    gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    
    # Create a chain with the prompt template and Gemini model
    
    chain = LLMChain(llm=gemini, prompt=search_prompt_template)
    
    # Run the chain with the search results

    summary = chain.run(topic=query, results=formatted_results)
    
    return summary, formatted_results 


Role: This function combines search and summarization:

->Calls safe_search_duckduckgo to get search results<br>
->Formats the results using format_search_results<br>
->Initializes the Gemini model<br>
->Creates a LangChain chain with the search prompt template<br>
->Runs the chain to generate a summary of the search results<br>
->Returns both the summary and the formatted results (to maintain context for follow-ups)<br>

## Function: handle_follow_up <br>
def handle_follow_up(follow_up_question, original_query, previous_summary, previous_results):<br>
    """<br>
    Handle follow-up questions by searching again if needed and using context.<br>
    """<br>
    # First, try to see if we need new information<br>
    combined_query = f"{original_query} {follow_up_question}"<br>
    
    # Do a new search for the follow-up 

    new_results = safe_search_duckduckgo(combined_query)

    formatted_new_results = format_search_results(new_results)
    
    # Initialize the Gemini model

    gemini = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    
    # Create a chain with the follow-up prompt template

    follow_up_chain = LLMChain(llm=gemini, prompt=follow_up_prompt_template)
    
    # Run the chain with all available context
    
    response = follow_up_chain.run(
        original_topic=original_query,
        follow_up_question=follow_up_question,
        previous_summary=previous_summary,
        new_results=formatted_new_results
    )
    
    return response, formatted_new_results

Role: This function processes follow-up questions by:

->Creating a combined query from the original query and follow-up question<br>
->Performing a new search with this combined query<br>
->Initializing the Gemini model<br>
->Creating a LangChain chain with the follow-up prompt template<br>
->Running the chain with all available context (original topic, follow-up question, previous summary, and new results)<br>
->Returning the response and the new formatted results<br>

## Function: conversational_handler <br>
def conversational_handler(): <br>
    """<br>
    Main function to handle the conversation flow.<br>
    """<br>
    print("Welcome to the Conversational Search System!")<br>
    print("Ask a question, and I'll search for information and summarize it.")<br>
    print("Type 'exit' at any time to quit.\n")<br>
    
    original_query = None

    previous_summary = None
    
    previous_results = None
    
    while True:

        # First query or new topic
        
        if not original_query:

            query = input("\nWhat would you like to search for? ")
            
            if query.lower() == 'exit':
                break
                
            print("\nSearching and summarizing...")

            summary, results = duckduckgo_search_and_summarize(query)
            
            print("\n--- SEARCH SUMMARY ---")

            print(summary)
            
            original_query = query

            previous_summary = summary

            previous_results = results
        
        # Follow-up questions

        follow_up = input("\nWhat else would you like to know about this topic? (or type 'new' for a new search, 'exit' to quit) ")
        
        if follow_up.lower() == 'exit':
            break
        elif follow_up.lower() == 'new':

            original_query = None

            previous_summary = None

            previous_results = None

            continue
        
        print("\nProcessing your follow-up question...")

        follow_up_response, new_results = handle_follow_up(
            follow_up, 
            original_query, 
            previous_summary, 
            previous_results
        )
        
        print("\n--- FOLLOW-UP RESPONSE ---")
        
        print(follow_up_response)
        
        # Update the previous summary to include this follow-up

        previous_summary += f"\n\nFollow-up about {follow_up}:\n{follow_up_response}"

        previous_results += new_results

Role: This is the main function that orchestrates the entire conversational experience:

->Welcomes the user and explains how to use the system<br>
->Initializes variables to track conversation state<br>
->Enters a loop to handle interactions<br>
->For initial queries:<br>
    ->Gets user input<br>
    ->Checks for exit command<br>
    ->Calls duckduckgo_search_and_summarize<br>
    ->Displays the summary<br>
    ->Stores the query, summary, and results for context<br>


->For follow-up queries:<br>
    ->Gets user input<br>
    ->Handles exit/new commands<br>
    ->Calls handle_follow_up with the stored context<br>
    ->Displays the response<br>
    ->Updates the stored context with the new information<br>
->Allows the user to start new topics or exit at any time

## Main Block <br>
if __name__ == "__main__":<br>
    conversational_handler()

Role: This ensures that the conversational_handler function is called when the script is run directly (not imported).<br>
## Overall Architecture and Flow

The system follows this flow:

1.User enters an initial query <br>
2.System searches DuckDuckGo and summarizes results with Gemini<br>
3.User asks follow-up questions<br>
4.System uses context from previous interactions plus new searches to answer<br>
5.Conversation continues until user exits or starts a new topic

The key innovation here is maintaining state between interactions, allowing for a more natural conversation that builds upon previous context. This makes the system more intuitive than having to restate the entire query with each interaction.